In [1]:
import pickle as pkl
data_path = 'data.pkl'
with open(data_path, 'rb') as f:
    batch = pkl.load(f)

In [2]:
from models.CMA import LSTCWA, MultiDimAligner
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
D_TARGET   = 512
NUM_TOKENS = 64
WIN        = 64
STRIDE     = 32

aligner   = MultiDimAligner(D_TARGET).to(device)
compressor= LSTCWA(dim=D_TARGET, num_tokens=NUM_TOKENS,
                    win_size=WIN, stride=STRIDE).to(device)

# --------- 单 batch 推理示例 ---------
#compressed_batch = []
# 
# # 每张 WSI 输出一个 [L, D] token
tokens_per_backbone = []
for feats_list, coords_list, pad_list in zip(batch['imgs_list'],
                                            batch['coords_list'],
                                            batch['pad_mask_list']):
    """注意：
       - feats_list  : List[Tensor]，长度 = backbone 数
         每个 tensor 形状 [T_i, D_in_i]
       - coords_list : List[Tensor]，同上
       - pad_list    : List[BoolTensor]，同上
    """
    
    feats = feats.to(device)          # [T, D_in]
    coords= coords.to(device)
    pad   = pad.to(device)
    feats_ali = aligner(feats)        # → [T, D_TARGET]
    Z = compressor(feats_ali, coords, pad)  # [L, D_TARGET]
    tokens_per_backbone.append(Z)




In [25]:
for i in tokens_per_backbone:
    print(i.shape)

torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])


In [2]:
import os
import shutil

def delete_folders_by_keyword(root_path, keyword):
    """
    递归删除 root_path 下所有名称包含 keyword 的子文件夹
    :param root_path: 要搜索的根目录
    :param keyword: 要匹配的关键词（区分大小写）
    """
    for root, dirs, files in os.walk(root_path, topdown=False):
        for dir_name in dirs:
            if keyword in dir_name:
                folder_path = os.path.join(root, dir_name)
                try:
                    print(f"正在删除文件夹: {folder_path}")
                    shutil.rmtree(folder_path)  # 递归删除整个文件夹
                except Exception as e:
                    print(f"删除失败: {folder_path} - {e}")

delete_folders_by_keyword('/home/yuhaowang/project/CMA/CMA-MOE/output_2','moe')

正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/AIDPATH_GRADE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/DORID_6/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/BCNB_ALN/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/BRACS_COARSE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/SLNBREAST_SUBTYPE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/TCGA-BRCA-SUBTYPE/moe


In [5]:
import os

def delete_files_by_suffix_recursive(directory: str, suffix: str):
    """
    递归删除指定目录及其所有子目录下，以指定后缀结尾的文件。

    参数:
        directory (str): 要处理的根目录路径。
        suffix (str): 要删除文件的后缀名，如 '.txt' 或 '.log'。
    """
    if not os.path.isdir(directory):
        print(f"路径无效：{directory}")
        return

    deleted_count = 0

    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(suffix):
                file_path = os.path.join(root, filename)
                try:
                    os.remove(file_path)
                    print(f"已删除文件: {file_path}")
                    deleted_count += 1
                except Exception as e:
                    print(f"无法删除文件 {file_path}：{e}")

    print(f"共删除 {deleted_count} 个后缀为 {suffix} 的文件。")

    
delete_files_by_suffix_recursive('/home/yuhaowang/project/CMA/CMA-MOE/output/','.pt')

已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/Gigapath/MIL/0.001/fold_2/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/Gigapath/MIL/0.001/fold_3/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/Gigapath/MIL/0.001/fold_1/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/Gigapath/MIL/0.001/fold_0/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/Gigapath/MIL/0.001/fold_4/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/UNI/MIL/0.001/fold_2/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/UNI/MIL/0.001/fold_3/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/UNI/MIL/0.001/fold_1/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/UNI/MIL/0.001/fold_0/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/output/BCNB_ER/UNI/MIL/0.001/fold_4/checkpoint.pt
已删除文件: /home/yuhaowang/project/CMA/CMA-MOE/ou

In [6]:
import os
import glob
import pandas as pd

# 设定你的实验结果的根目录
base_path = '/home/yuhaowang/project/CMA/CMA-MOE/output'  # 请替换为实际的路径

# 获取所有任务目录
tasks = os.listdir(base_path)

results_dict = {}

# 遍历每个任务和模型，递归提取summary.csv
for task in tasks:
    task_path = os.path.join(base_path, task)
    model_dirs = os.listdir(task_path)
    
    for model in model_dirs:
        model_path = os.path.join(task_path, model)
        summary_files = glob.glob(os.path.join(model_path, '**', 'summary.csv'), recursive=True)

        auroc_values = []

        for file in summary_files:
            df = pd.read_csv(file)
            if 'val_macro_auroc' in df.columns:
                auroc_values.extend(df['val_macro_auroc'].values)

        if auroc_values:
            mean_auroc = pd.Series(auroc_values).mean()
            std_auroc = pd.Series(auroc_values).std()
            
            if model not in results_dict:
                results_dict[model] = {}

            results_dict[model][task] = f"{mean_auroc:.3f}±{std_auroc:.3f}"

# 转换为DataFrame并整理
results_df = pd.DataFrame(results_dict).T

# 将'moe'放到第一行（如果存在的话）
if 'moe' in results_df.index:
    results_df = pd.concat([results_df.loc[['moe']], results_df.drop('moe')])

# 输出结果为CSV文件
output_csv_path = 'aggregated_results.csv'  # 可自行更改保存路径
results_df.to_csv(output_csv_path)

print(f"实验结果已成功保存到 {output_csv_path}")


实验结果已成功保存到 aggregated_results.csv
